# 주중 주말 공휴일 데이터

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

- 20031201 - 20221231 데이터프레임 생성

In [12]:
pd.date_range("20031201", "20221231")

DatetimeIndex(['2003-12-01', '2003-12-02', '2003-12-03', '2003-12-04',
               '2003-12-05', '2003-12-06', '2003-12-07', '2003-12-08',
               '2003-12-09', '2003-12-10',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=6971, freq='D')

In [201]:
date_df = pd.DataFrame(data=pd.date_range("20031201", "20221231"), index=range(6971))

In [202]:
date_df.rename(columns={0 : "date"}, inplace=True)
date_df.head()

,date
0,2003-12-01
1,2003-12-02
2,2003-12-03
3,2003-12-04
4,2003-12-05


- 평일 주말 담을 컬럼 생성

In [203]:
date_df["week"] = "-"
date_df["day"] = "-"
date_df

,date,week,day
0,2003-12-01,-,-
1,2003-12-02,-,-
2,2003-12-03,-,-
3,2003-12-04,-,-
4,2003-12-05,-,-
...,...,...,...
6966,2022-12-27,-,-
6967,2022-12-28,-,-
6968,2022-12-29,-,-
6969,2022-12-30,-,-


- weekday함수로 평일 주말 구분해서 값 넣기

In [204]:
idx = 0
for dt in date_df["date"]:
    weekday = dt.weekday()
    if weekday <= 4:
        date_df["week"][idx] = 0
    else:
        date_df["week"][idx] = 1
    idx += 1

C:\Users\eunga\AppData\Local\Temp\ipykernel_18300\3076567572.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df["week"][idx] = 0
C:\Users\eunga\AppData\Local\Temp\ipykernel_18300\3076567572.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df["week"][idx] = 1


In [205]:
## 요일 넣기 ##
date_df['day'] = pd.to_datetime(date_df["date"],format="%Y-%m-%d").dt.strftime('%a')

In [206]:
date_df

,date,week,day
0,2003-12-01,0,Mon
1,2003-12-02,0,Tue
2,2003-12-03,0,Wed
3,2003-12-04,0,Thu
4,2003-12-05,0,Fri
...,...,...,...
6966,2022-12-27,0,Tue
6967,2022-12-28,0,Wed
6968,2022-12-29,0,Thu
6969,2022-12-30,0,Fri


In [207]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6971 entries, 0 to 6970
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6971 non-null   datetime64[ns]
 1   week    6971 non-null   object        
 2   day     6971 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 163.5+ KB


In [208]:
## check point ##
date_df.to_csv("date.csv", encoding="utf-8", index=False)

In [10]:
date_df = pd.read_csv("date.csv")

In [11]:
date_df

,date,week,day
0,2003-12-01,0,Mon
1,2003-12-02,0,Tue
2,2003-12-03,0,Wed
3,2003-12-04,0,Thu
4,2003-12-05,0,Fri
...,...,...,...
6966,2022-12-27,0,Tue
6967,2022-12-28,0,Wed
6968,2022-12-29,0,Thu
6969,2022-12-30,0,Fri


------
- 공휴일 찾기
    - API출처 : https://www.data.go.kr/data/15012690/openapi.do

In [1]:
## 2004 ~ 2022년도 공휴일 수집 ##
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse as urlparse


def get_request_query(url, operation, params, serviceKey):
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

mykey = "ApRZ95DtQHw7scnx1D%2FHIOoxouuIvvx%2BSm86AbFRaJS4WqS7EaGQASlMbd0y7aLAuuuFla867EYaGau0blJFIA%3D%3D"
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
operation = 'getRestDeInfo'

df = pd.DataFrame()

for yr in range(2002, 2023):
    for month in range(1,13):

        if month < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        params = {'solYear':yr, 'solMonth':month}

        request_query = get_request_query(url, operation, params, mykey)

        res = requests.get(request_query)
        soup = BeautifulSoup(res.text, 'lxml')
        items = soup.find_all('item')

        for item in items:
            day = item.locdate.get_text()
            name = item.datename.get_text()
            data = {'holiday':name , 'date':day}
            df = df.append(data, ignore_index=True)
    
display(df.head())
display(df.tail())

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C

C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter 

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter 

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter 

...

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C

C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\1420947394.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\eunga\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter 

,holiday,date
0,신정,20040101
1,설날,20040121
2,설날,20040122
3,설날,20040123
4,삼일절,20040301


,holiday,date
303,대체공휴일,20220912
304,개천절,20221003
305,한글날,20221009
306,대체공휴일,20221010
307,기독탄신일,20221225


In [2]:
df

,holiday,date
0,신정,20040101
1,설날,20040121
2,설날,20040122
3,설날,20040123
4,삼일절,20040301
...,...,...
303,대체공휴일,20220912
304,개천절,20221003
305,한글날,20221009
306,대체공휴일,20221010


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   holiday  308 non-null    object
 1   date     308 non-null    object
dtypes: object(2)
memory usage: 4.9+ KB


In [4]:
## 날짜형식으로 맞춰주기 ##
from datetime import datetime

idx = 0
for d in df["date"]:
    df["date"][idx] = datetime.strptime(str(df["date"][idx]), "%Y%m%d")
    idx += 1

In [5]:
df["date"]

0      2004-01-01 00:00:00
1      2004-01-21 00:00:00
2      2004-01-22 00:00:00
3      2004-01-23 00:00:00
4      2004-03-01 00:00:00
              ...         
303    2022-09-12 00:00:00
304    2022-10-03 00:00:00
305    2022-10-09 00:00:00
306    2022-10-10 00:00:00
307    2022-12-25 00:00:00
Name: date, Length: 308, dtype: object

In [6]:
df["date"] = df["date"].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

In [7]:
df["date"] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [13]:
date_df["date"] = date_df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holiday  308 non-null    object        
 1   date     308 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 4.9+ KB


In [224]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6971 entries, 0 to 6970
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6971 non-null   datetime64[ns]
 1   week    6971 non-null   int64         
 2   day     6971 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 163.5+ KB


In [232]:
## check point ##
df.to_csv("holiday.csv", encoding="utf-8", index=False)

------
- date_df에 공휴일 정보 합치기

In [21]:
df.head(3)

,holiday,date
0,신정,2004-01-01
1,설날,2004-01-21
2,설날,2004-01-22


In [43]:
date_df.head(3)

,date,week,day
0,2003-12-01,0,Mon
1,2003-12-02,0,Tue
2,2003-12-03,0,Wed


In [14]:
## 데이터 프레임 합치기 ##
all_date = pd.merge(date_df, df, how='left')

In [15]:
## 공휴일 결측치 0으로 채우기 ##
all_date["holiday"] = all_date["holiday"].fillna(0)

In [16]:
## check point ##
all_date.to_csv("all_date.csv", encoding="utf-8", index=False)

PermissionError: [Errno 13] Permission denied: 'all_date.csv'

In [17]:
## 공휴일인 날을 1로 바꾸기 ##
for i in range(len(all_date["holiday"])):
    if all_date["holiday"][i] != 0:
        all_date["holiday"][i] = 1

C:\Users\eunga\AppData\Local\Temp\ipykernel_22088\528331193.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_date["holiday"][i] = 1


In [18]:
## 형식 변환 ##
all_date["holiday"] = pd.to_numeric(all_date["holiday"])

In [19]:
all_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6974 entries, 0 to 6973
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     6974 non-null   datetime64[ns]
 1   week     6974 non-null   int64         
 2   day      6974 non-null   object        
 3   holiday  6974 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 530.5+ KB


In [23]:
all_date.value_counts('date')

date
2006-05-05    2
2009-10-03    2
2017-10-03    2
2016-08-16    1
2016-08-27    1
             ..
2010-04-10    1
2010-04-09    1
2010-04-08    1
2010-04-07    1
2022-12-31    1
Length: 6971, dtype: int64

In [39]:
all_date.loc[all_date['date'] == '2006-05-05', :] # 887 행

,date,week,day,holiday
886,2006-05-05,0,Fri,1


In [42]:
all_date.loc[886]

date       2006-05-05 00:00:00
week                         0
day                        Fri
holiday                      1
Name: 886, dtype: object

In [36]:
all_date.loc[all_date['date'] == '2009-10-03', :] # 2134

,date,week,day,holiday
2134,2009-10-03,1,Sat,1
2135,2009-10-03,1,Sat,1


In [38]:
all_date.loc[all_date['date'] == '2017-10-03', :]

,date,week,day,holiday
5057,2017-10-03,0,Tue,1
5058,2017-10-03,0,Tue,1


In [33]:
all_date.drop(887,inplace=True)

In [243]:
all_date.to_csv("all_date.csv", encoding="utf-8", index=False)